In [1]:
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns 
import numpy as np

# modelization 
import sklearn
from sklearn import model_selection
from sklearn import preprocessing
from sklearn import linear_model
from sklearn.model_selection import cross_val_score
## system
import glob, os

In [2]:
def Read_Parquet(chemin):
  return pd.read_parquet(chemin, engine='fastparquet')

In [3]:
train_path = "C:/Users/alaek/DEFI_IA/france_meteo/Défi_IA/input/imputed_data/X_Train_concat_y_train.gzip"
test_path = "C:/Users/alaek/DEFI_IA/france_meteo/Défi_IA/input/imputed_data/X_Test_merged_clean.gzip"
base_for_test_path = "C:/Users/alaek/DEFI_IA/france_meteo/Défi_IA/input/Test/Baselines/Baseline_forecast_test.csv"
base_for_train_path = "C:/Users/alaek/DEFI_IA/france_meteo/Défi_IA/input/Train/Baselines/Baseline_forecast_train.csv"

In [4]:
train = Read_Parquet(train_path)
train.reset_index(inplace=True)
train.rename(columns={'Index':'Id'}, inplace=True)
train.head()

,Id,temp_min,temp_max,temp_mean,temp_median,temp_percentile_25,temp_percentile_75,temp_skew,temp_kurtosis,humidity_min,...,precip_percentile_25,precip_percentile_75,precip_skew,precip_kurtosis,Month,season,lat,lon,height_sta,Ground_truth
0,14066001_0,276.91,282.59,280.333750,281.325,278.5100,282.1750,-0.445114,-1.418811,79.9,...,0.0,0.00,4.587317,19.043478,1,winter,49.334,-0.431,2.0,3.4
1,14066001_1,280.65,284.37,282.936667,282.870,282.1925,283.9625,-0.204054,-0.728304,71.8,...,0.0,0.05,2.768227,7.524247,1,winter,49.334,-0.431,2.0,11.7
2,14066001_10,277.91,281.33,279.557917,279.520,279.1575,280.1750,0.097627,-0.208982,81.4,...,0.0,0.40,2.517940,5.760873,1,winter,49.334,-0.431,2.0,1.0
3,14066001_100,275.53,287.57,282.112917,283.950,276.8575,285.5100,-0.439847,-1.378881,56.1,...,0.0,0.00,4.410995,17.920294,4,winter,49.334,-0.431,2.0,5.6
4,14066001_101,279.29,285.41,282.805000,282.720,282.2225,284.0125,-0.378186,-0.411371,76.8,...,0.0,0.00,2.538396,5.073702,4,winter,49.334,-0.431,2.0,3.2


In [5]:
base_for_train = pd.read_csv(base_for_train_path)
base_for_train.drop(['number_sta','date'], axis=1, inplace=True)
base_for_train.rename(columns={'Prediction':'Prediction_meteo_france'}, inplace=True)
base_for_train.head()

,Prediction_meteo_france,Id
0,5.027973,86118001_0
1,0.957647,86149001_0
2,21.019514,56081003_0
3,4.260686,53215001_0
4,17.256493,22135001_0


In [6]:
base_for_test = pd.read_csv(base_for_test_path)
base_for_test.rename(columns={'Prediction':'Prediction_meteo_france'}, inplace=True)
base_for_test.head()

,Prediction_meteo_france,Id
0,12.921875,86118001_149
1,2.186354,86149001_149
2,0.036655,56081003_149
3,0.024249,53215001_149
4,0.377269,22135001_149


In [7]:
test = Read_Parquet(test_path)
test.rename(columns={'Index':'Id'}, inplace=True)
test.head()

,Id,temp_min,temp_max,temp_mean,temp_median,temp_percentile_25,temp_percentile_75,temp_skew,temp_kurtosis,humidity_min,...,precip_median,precip_percentile_25,precip_percentile_75,precip_skew,precip_kurtosis,Month,season,lat,lon,height_sta
index,,,,,,,,,,,,,,,,,,,,,
0,14047002_100,281.86,283.24,282.565000,282.515,282.2875,282.835,-0.000790,-0.932193,88.36,...,0.000,0.0,0.45,1.483913,0.656080,12,winter,49.275,-0.712,60.0
1,14047002_223,283.69,286.43,285.415000,285.400,284.9400,285.905,-0.519703,-0.350026,75.69,...,0.000,0.0,0.00,3.989829,14.892857,12,winter,49.275,-0.712,60.0
2,14047002_236,282.42,284.47,283.064167,282.990,282.6450,283.215,1.358096,1.388465,79.28,...,0.000,0.0,0.30,2.813251,8.290722,12,winter,49.275,-0.712,60.0
3,14047002_256,278.88,286.40,283.611250,284.625,282.3075,285.575,-0.792405,-0.848946,77.21,...,0.000,0.0,0.00,2.514334,5.405663,12,winter,49.275,-0.712,60.0
4,14047002_277,278.35,283.85,282.006000,283.040,280.4550,283.405,-0.869940,-0.830229,87.36,...,0.165,0.0,0.65,1.414487,0.573291,12,winter,49.275,-0.712,60.0


In [8]:
test= pd.merge(base_for_test, test, how='right').set_index("Id")
test.head()

,Prediction_meteo_france,temp_min,temp_max,temp_mean,temp_median,temp_percentile_25,temp_percentile_75,temp_skew,temp_kurtosis,humidity_min,...,precip_median,precip_percentile_25,precip_percentile_75,precip_skew,precip_kurtosis,Month,season,lat,lon,height_sta
Id,,,,,,,,,,,,,,,,,,,,,
14047002_100,4.361040,281.86,283.24,282.565000,282.515,282.2875,282.835,-0.000790,-0.932193,88.36,...,0.000,0.0,0.45,1.483913,0.656080,12,winter,49.275,-0.712,60.0
14047002_223,0.000000,283.69,286.43,285.415000,285.400,284.9400,285.905,-0.519703,-0.350026,75.69,...,0.000,0.0,0.00,3.989829,14.892857,12,winter,49.275,-0.712,60.0
14047002_236,4.294544,282.42,284.47,283.064167,282.990,282.6450,283.215,1.358096,1.388465,79.28,...,0.000,0.0,0.30,2.813251,8.290722,12,winter,49.275,-0.712,60.0
14047002_256,8.192842,278.88,286.40,283.611250,284.625,282.3075,285.575,-0.792405,-0.848946,77.21,...,0.000,0.0,0.00,2.514334,5.405663,12,winter,49.275,-0.712,60.0
14047002_277,7.473648,278.35,283.85,282.006000,283.040,280.4550,283.405,-0.869940,-0.830229,87.36,...,0.165,0.0,0.65,1.414487,0.573291,12,winter,49.275,-0.712,60.0


In [9]:
train= pd.merge(base_for_train, train, how='right').set_index("Id")
train.head()

,Prediction_meteo_france,temp_min,temp_max,temp_mean,temp_median,temp_percentile_25,temp_percentile_75,temp_skew,temp_kurtosis,humidity_min,...,precip_percentile_25,precip_percentile_75,precip_skew,precip_kurtosis,Month,season,lat,lon,height_sta,Ground_truth
Id,,,,,,,,,,,,,,,,,,,,,
14066001_0,5.545129,276.91,282.59,280.333750,281.325,278.5100,282.1750,-0.445114,-1.418811,79.9,...,0.0,0.00,4.587317,19.043478,1,winter,49.334,-0.431,2.0,3.4
14066001_1,7.372426,280.65,284.37,282.936667,282.870,282.1925,283.9625,-0.204054,-0.728304,71.8,...,0.0,0.05,2.768227,7.524247,1,winter,49.334,-0.431,2.0,11.7
14066001_10,7.151256,277.91,281.33,279.557917,279.520,279.1575,280.1750,0.097627,-0.208982,81.4,...,0.0,0.40,2.517940,5.760873,1,winter,49.334,-0.431,2.0,1.0
14066001_100,1.914063,275.53,287.57,282.112917,283.950,276.8575,285.5100,-0.439847,-1.378881,56.1,...,0.0,0.00,4.410995,17.920294,4,winter,49.334,-0.431,2.0,5.6
14066001_101,0.581547,279.29,285.41,282.805000,282.720,282.2225,284.0125,-0.378186,-0.411371,76.8,...,0.0,0.00,2.538396,5.073702,4,winter,49.334,-0.431,2.0,3.2


In [10]:
## Encoding test 
train= pd.get_dummies(train,prefix_sep='_')
train.head()

,Prediction_meteo_france,temp_min,temp_max,temp_mean,temp_median,temp_percentile_25,temp_percentile_75,temp_skew,temp_kurtosis,humidity_min,...,Month_7,Month_8,Month_9,Month_10,Month_11,Month_12,season_fall,season_spring,season_summer,season_winter
Id,,,,,,,,,,,,,,,,,,,,,
14066001_0,5.545129,276.91,282.59,280.333750,281.325,278.5100,282.1750,-0.445114,-1.418811,79.9,...,0,0,0,0,0,0,0,0,0,1
14066001_1,7.372426,280.65,284.37,282.936667,282.870,282.1925,283.9625,-0.204054,-0.728304,71.8,...,0,0,0,0,0,0,0,0,0,1
14066001_10,7.151256,277.91,281.33,279.557917,279.520,279.1575,280.1750,0.097627,-0.208982,81.4,...,0,0,0,0,0,0,0,0,0,1
14066001_100,1.914063,275.53,287.57,282.112917,283.950,276.8575,285.5100,-0.439847,-1.378881,56.1,...,0,0,0,0,0,0,0,0,0,1
14066001_101,0.581547,279.29,285.41,282.805000,282.720,282.2225,284.0125,-0.378186,-0.411371,76.8,...,0,0,0,0,0,0,0,0,0,1


In [11]:
## Encoding test 
test = pd.get_dummies(test,prefix_sep='_')
test.head()

,Prediction_meteo_france,temp_min,temp_max,temp_mean,temp_median,temp_percentile_25,temp_percentile_75,temp_skew,temp_kurtosis,humidity_min,...,Month_7,Month_8,Month_9,Month_10,Month_11,Month_12,season_fall,season_spring,season_summer,season_winter
Id,,,,,,,,,,,,,,,,,,,,,
14047002_100,4.361040,281.86,283.24,282.565000,282.515,282.2875,282.835,-0.000790,-0.932193,88.36,...,0,0,0,0,0,1,0,0,0,1
14047002_223,0.000000,283.69,286.43,285.415000,285.400,284.9400,285.905,-0.519703,-0.350026,75.69,...,0,0,0,0,0,1,0,0,0,1
14047002_236,4.294544,282.42,284.47,283.064167,282.990,282.6450,283.215,1.358096,1.388465,79.28,...,0,0,0,0,0,1,0,0,0,1
14047002_256,8.192842,278.88,286.40,283.611250,284.625,282.3075,285.575,-0.792405,-0.848946,77.21,...,0,0,0,0,0,1,0,0,0,1
14047002_277,7.473648,278.35,283.85,282.006000,283.040,280.4550,283.405,-0.869940,-0.830229,87.36,...,0,0,0,0,0,1,0,0,0,1


In [12]:
train.isna().sum()

Prediction_meteo_france    7429
temp_min                      0
temp_max                      0
temp_mean                     0
temp_median                   0
                           ... 
Month_12                      0
season_fall                   0
season_spring                 0
season_summer                 0
season_winter                 0
Length: 70, dtype: int64

In [13]:
# values = train[train['Prediction_meteo_france'].isna()]['Ground_truth'].values
# list(values)

In [14]:
# dictionnary

In [15]:
train.fillna(train.median(), inplace=True)

In [16]:
test.fillna(test.median(), inplace=True)

In [17]:
# train.fillna(value={'Prediction_meteo_france': list(values)})

## Feature Engineering 

### Normalization

In [18]:
Features_to_normalize = ['temp_min', 'temp_max', 'temp_mean',
       'temp_median', 'temp_percentile_25', 'temp_percentile_75', 'temp_skew',
       'temp_kurtosis', 'humidity_min', 'humidity_max', 'humidity_mean',
       'humidity_median', 'humidity_percentile_25', 'humidity_percentile_75',
       'humidity_skew', 'humidity_kurtosis', 'dew_point_temp_min',
       'dew_point_temp_max', 'dew_point_temp_mean', 'dew_point_temp_median',
       'dew_point_temp_percentile_25', 'dew_point_temp_percentile_75',
       'dew_point_temp_skew', 'dew_point_temp_kurtosis', 'Vx_min', 'Vx_max',
       'Vx_mean', 'Vx_median', 'Vx_percentile_25', 'Vx_percentile_75',
       'Vx_skew', 'Vx_kurtosis', 'Vy_min', 'Vy_max', 'Vy_mean', 'Vy_median',
       'Vy_percentile_25', 'Vy_percentile_75', 'Vy_skew', 'Vy_kurtosis',
       'precip_sum', 'precip_min', 'precip_max', 'precip_mean',
       'precip_median', 'precip_percentile_25', 'precip_percentile_75',
       'precip_skew', 'precip_kurtosis']

In [19]:
from sklearn import preprocessing
# normalize the data attributes
train[Features_to_normalize] = preprocessing.normalize(train[Features_to_normalize])
train.head()

,Prediction_meteo_france,temp_min,temp_max,temp_mean,temp_median,temp_percentile_25,temp_percentile_75,temp_skew,temp_kurtosis,humidity_min,...,Month_7,Month_8,Month_9,Month_10,Month_11,Month_12,season_fall,season_spring,season_summer,season_winter
Id,,,,,,,,,,,,,,,,,,,,,
14066001_0,5.545129,0.279014,0.284737,0.282464,0.283463,0.280626,0.284319,-0.000448,-0.001430,0.080507,...,0,0,0,0,0,0,0,0,0,1
14066001_1,7.372426,0.281820,0.285556,0.284117,0.284050,0.283369,0.285147,-0.000205,-0.000731,0.072099,...,0,0,0,0,0,0,0,0,0,1
14066001_10,7.151256,0.281175,0.284635,0.282842,0.282804,0.282437,0.283467,0.000099,-0.000211,0.082356,...,0,0,0,0,0,0,0,0,0,1
14066001_100,1.914063,0.278868,0.291054,0.285531,0.287390,0.280212,0.288969,-0.000445,-0.001396,0.056780,...,0,0,0,0,0,0,0,0,0,1
14066001_101,0.581547,0.279321,0.285442,0.282836,0.282751,0.282254,0.284044,-0.000378,-0.000411,0.076809,...,0,0,0,0,0,0,0,0,0,1


In [20]:
test[Features_to_normalize] = preprocessing.normalize(test[Features_to_normalize])
test.head()

,Prediction_meteo_france,temp_min,temp_max,temp_mean,temp_median,temp_percentile_25,temp_percentile_75,temp_skew,temp_kurtosis,humidity_min,...,Month_7,Month_8,Month_9,Month_10,Month_11,Month_12,season_fall,season_spring,season_summer,season_winter
Id,,,,,,,,,,,,,,,,,,,,,
14047002_100,4.361040,0.280756,0.282131,0.281459,0.281409,0.281182,0.281728,-7.870904e-07,-0.000929,0.088014,...,0,0,0,0,0,1,0,0,0,1
14047002_223,0.000000,0.281923,0.284646,0.283637,0.283622,0.283165,0.284124,-5.164654e-04,-0.000348,0.075218,...,0,0,0,0,0,1,0,0,0,1
14047002_236,4.294544,0.282810,0.284863,0.283455,0.283381,0.283036,0.283607,1.359973e-03,0.001390,0.079390,...,0,0,0,0,0,1,0,0,0,1
14047002_256,8.192842,0.278335,0.285841,0.283057,0.284069,0.281756,0.285017,-7.908581e-04,-0.000847,0.077059,...,0,0,0,0,0,1,0,0,0,1
14047002_277,7.473648,0.278377,0.283877,0.282033,0.283067,0.280482,0.283432,-8.700238e-04,-0.000830,0.087368,...,0,0,0,0,0,1,0,0,0,1


In [21]:
import ppscore as pps
pps.predictors(train, "Ground_truth", sorted=True).set_index('ppscore').sort_index(ascending=False)[['x','y','model_score','baseline_score','metric']]

,x,y,model_score,baseline_score,metric
ppscore,,,,,
0.07345,Prediction_meteo_france,Ground_truth,1.792114,1.93418,mean absolute error
0.00000,temp_min,Ground_truth,3.236127,1.93418,mean absolute error
0.00000,temp_max,Ground_truth,3.145560,1.93418,mean absolute error
0.00000,temp_mean,Ground_truth,3.151563,1.93418,mean absolute error
0.00000,temp_median,Ground_truth,3.129693,1.93418,mean absolute error
...,...,...,...,...,...
0.00000,Month_12,Ground_truth,2.634723,1.93418,mean absolute error
0.00000,season_fall,Ground_truth,2.627843,1.93418,mean absolute error
0.00000,season_spring,Ground_truth,2.632250,1.93418,mean absolute error


In [22]:
pd.DataFrame(np.abs(train.corr()['Ground_truth'])).sort_values(by='Ground_truth', axis=0, ascending=False)

,Ground_truth
Ground_truth,1.000000
Prediction_meteo_france,0.734291
Vx_min,0.263692
Vx_percentile_25,0.252869
Vx_mean,0.244072
...,...
dew_point_temp_mean,0.006285
dew_point_temp_median,0.005749
Vy_kurtosis,0.003354
Month_6,0.001927


In [23]:
train.dtypes

Prediction_meteo_france    float64
temp_min                   float64
temp_max                   float64
temp_mean                  float64
temp_median                float64
                            ...   
Month_12                     uint8
season_fall                  uint8
season_spring                uint8
season_summer                uint8
season_winter                uint8
Length: 70, dtype: object

## Polynomial feature engineering

In [24]:
train.reset_index(inplace=True)

In [25]:
test.reset_index(inplace=True)

In [26]:
## modelization 
import sklearn
from sklearn import model_selection
from sklearn import linear_model
from sklearn.model_selection import cross_val_score
## system
import glob, os

In [27]:
features_eng = ['Prediction_meteo_france', 'temp_min', 'temp_max', 'temp_mean',
       'temp_median', 'temp_percentile_25', 'temp_percentile_75', 'temp_skew',
       'temp_kurtosis', 'humidity_min', 'humidity_max', 'humidity_mean',
       'humidity_median', 'humidity_percentile_25', 'humidity_percentile_75',
       'humidity_skew', 'humidity_kurtosis', 'dew_point_temp_min',
       'dew_point_temp_max', 'dew_point_temp_mean', 'dew_point_temp_median',
       'dew_point_temp_percentile_25', 'dew_point_temp_percentile_75',
       'dew_point_temp_skew', 'dew_point_temp_kurtosis', 'Vx_min', 'Vx_max',
       'Vx_mean', 'Vx_median', 'Vx_percentile_25', 'Vx_percentile_75',
       'Vx_skew', 'Vx_kurtosis', 'Vy_min', 'Vy_max', 'Vy_mean', 'Vy_median',
       'Vy_percentile_25', 'Vy_percentile_75', 'Vy_skew', 'Vy_kurtosis',
       'precip_sum', 'precip_min', 'precip_max', 'precip_mean',
       'precip_median', 'precip_percentile_25', 'precip_percentile_75',
       'precip_skew', 'precip_kurtosis']

In [28]:
features_stable = ['Id',	'Month_1',	'Month_2',	'Month_3',	'Month_4',	'Month_5',	'Month_6',	'Month_7',	'Month_8',	'Month_9',	'Month_10',	'Month_11',	'Month_12','Ground_truth',
 'season_fall','season_spring',	'season_summer','season_winter','lat','lon','height_sta']

In [29]:
def feature_engineering(scaled_kfolds_train, scaled_test,features):

    pf = preprocessing.PolynomialFeatures( 
    degree=2, 
    interaction_only=True, 
    include_bias=False )
    ## Fit the polynomial model for engineering 
    pf.fit(scaled_kfolds_train[features])
    ## Transform the training data feature
    poly_feats_train = pf.transform(scaled_kfolds_train[features])
    ## Trasform the test data features
    poly_feats_test = pf.transform(scaled_test[features])


    ## Rename the polynomial features with appropriate names 

    # create a dataframe with all the features of training val data
    num_feats = poly_feats_train.shape[1]
    print(f"number of feature created :{num_feats}") 
    df_transformed_train = pd.DataFrame( 
        poly_feats_train, 
        columns=[f"{i}" for i in pf.get_feature_names(input_features=scaled_kfolds_train[features].columns)] 
    )

    df_transformed_test = pd.DataFrame( 
        poly_feats_test, columns=[f"{i}" for i in pf.get_feature_names(input_features=scaled_test[features].columns)]) 

    return df_transformed_train, df_transformed_test

In [30]:
df_transformed_train, df_transformed_test = feature_engineering(train, test,features_eng)

number of feature created :1275


In [31]:
df_transformed_train.head()

,Prediction_meteo_france,temp_min,temp_max,temp_mean,temp_median,temp_percentile_25,temp_percentile_75,temp_skew,temp_kurtosis,humidity_min,...,precip_median precip_percentile_25,precip_median precip_percentile_75,precip_median precip_skew,precip_median precip_kurtosis,precip_percentile_25 precip_percentile_75,precip_percentile_25 precip_skew,precip_percentile_25 precip_kurtosis,precip_percentile_75 precip_skew,precip_percentile_75 precip_kurtosis,precip_skew precip_kurtosis
0,5.545129,0.279014,0.284737,0.282464,0.283463,0.280626,0.284319,-0.000448,-0.001430,0.080507,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000e+00,0.000000e+00,0.000089
1,7.372426,0.281820,0.285556,0.284117,0.284050,0.283369,0.285147,-0.000205,-0.000731,0.072099,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.395682e-07,3.793566e-07,0.000021
2,7.151256,0.281175,0.284635,0.282842,0.282804,0.282437,0.283467,0.000099,-0.000211,0.082356,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.030981e-06,2.358812e-06,0.000015
3,1.914063,0.278868,0.291054,0.285531,0.287390,0.280212,0.288969,-0.000445,-0.001396,0.056780,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000e+00,0.000000e+00,0.000081
4,0.581547,0.279321,0.285442,0.282836,0.282751,0.282254,0.284044,-0.000378,-0.000411,0.076809,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000e+00,0.000000e+00,0.000013


In [32]:
train_eng = pd.concat([train[features_stable],df_transformed_train],axis=1)
train_eng.head()

,Id,Month_1,Month_2,Month_3,Month_4,Month_5,Month_6,Month_7,Month_8,Month_9,...,precip_median precip_percentile_25,precip_median precip_percentile_75,precip_median precip_skew,precip_median precip_kurtosis,precip_percentile_25 precip_percentile_75,precip_percentile_25 precip_skew,precip_percentile_25 precip_kurtosis,precip_percentile_75 precip_skew,precip_percentile_75 precip_kurtosis,precip_skew precip_kurtosis
0,14066001_0,1,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000e+00,0.000000e+00,0.000089
1,14066001_1,1,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.395682e-07,3.793566e-07,0.000021
2,14066001_10,1,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.030981e-06,2.358812e-06,0.000015
3,14066001_100,0,0,0,1,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000e+00,0.000000e+00,0.000081
4,14066001_101,0,0,0,1,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000e+00,0.000000e+00,0.000013


In [33]:
features_stable_test =  ['Id',	'Month_1',	'Month_2',	'Month_3',	'Month_4',	'Month_5',	'Month_6',	'Month_7',	'Month_8',	'Month_9',	'Month_10',	'Month_11',	'Month_12',
 'season_fall','season_spring',	'season_summer',	'season_winter','lat','lon','height_sta']
test_eng = pd.concat([test[features_stable_test], df_transformed_test],axis=1)
test_eng.head()

,Id,Month_1,Month_2,Month_3,Month_4,Month_5,Month_6,Month_7,Month_8,Month_9,...,precip_median precip_percentile_25,precip_median precip_percentile_75,precip_median precip_skew,precip_median precip_kurtosis,precip_percentile_25 precip_percentile_75,precip_percentile_25 precip_skew,precip_percentile_25 precip_kurtosis,precip_percentile_75 precip_skew,precip_percentile_75 precip_kurtosis,precip_skew precip_kurtosis
0,14047002_100,0,0,0,0,0,0,0,0,0,...,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0.0,0.0,6.625416e-07,2.929287e-07,9.659567e-07
1,14047002_223,0,0,0,0,0,0,0,0,0,...,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0.0,0.0,0.000000e+00,0.000000e+00,5.868191e-05
2,14047002_236,0,0,0,0,0,0,0,0,0,...,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0.0,0.0,8.463105e-07,2.494098e-06,2.338842e-05
3,14047002_256,0,0,0,0,0,0,0,0,0,...,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0.0,0.0,0.000000e+00,0.000000e+00,1.353861e-05
4,14047002_277,0,0,0,0,0,0,0,0,0,...,0.0,1.072708e-07,2.334356e-07,9.461141e-08,0.0,0.0,0.0,9.195948e-07,3.727116e-07,8.110705e-07


## Feature selection 
### Lasso model

In [41]:
def feature_selection(df_train_eng):
    poly = df_train_eng
    X = poly.drop(["Id","Ground_truth"],axis=1).values
    y = poly["Ground_truth"].values
    lasso = linear_model.LassoCV(random_state=2021,cv=10,n_jobs=-1,max_iter=50000, selection='random').fit(X, y)
    importance = np.abs(lasso.coef_)
    dict_selection =dict()
    for k,j in zip([x for x in poly.drop(["Ground_truth", "Id"], axis=1)],importance):
        dict_selection[k]=j
    
   # threshold = np.mean(list(dict_selection.values()))
    threshold =0.0
    features_selected = []
    for i,j in dict_selection.items():
        if j >= threshold:
            features_selected.append(i)
    print(f"The number of final selection of features for our target is {len(features_selected)}")

    return features_selected, dict_selection, importance

In [42]:
features_selected, dict_selection, importance = feature_selection(train_eng)

The number of final selection of features for our target is 1294


In [44]:
dict(sorted(dict_selection.items(), key=lambda item: item[1], reverse=True))

{'Prediction_meteo_france': 0.7027968624022817,
 'Month_7': 0.07095713643085197,
 'season_summer': 0.06619345205309589,
 'lon': 0.06419689687906863,
 'lat': 0.03387181309406848,
 'season_winter': 0.03013447933707774,
 'Month_9': 0.006840223341317901,
 'Month_4': 0.0015848553423529206,
 'height_sta': 0.0003855449021581549,
 'Month_1': 0.0,
 'Month_2': 0.0,
 'Month_3': 0.0,
 'Month_5': 0.0,
 'Month_6': 0.0,
 'Month_8': 0.0,
 'Month_10': 0.0,
 'Month_11': 0.0,
 'Month_12': 0.0,
 'season_fall': 0.0,
 'season_spring': 0.0,
 'temp_min': 0.0,
 'temp_max': 0.0,
 'temp_mean': 0.0,
 'temp_median': 0.0,
 'temp_percentile_25': 0.0,
 'temp_percentile_75': 0.0,
 'temp_skew': 0.0,
 'temp_kurtosis': 0.0,
 'humidity_min': 0.0,
 'humidity_max': 0.0,
 'humidity_mean': 0.0,
 'humidity_median': 0.0,
 'humidity_percentile_25': 0.0,
 'humidity_percentile_75': 0.0,
 'humidity_skew': 0.0,
 'humidity_kurtosis': 0.0,
 'dew_point_temp_min': 0.0,
 'dew_point_temp_max': 0.0,
 'dew_point_temp_mean': 0.0,
 'dew_point

In [45]:
threshold =0
features_selected = []
for i,j in dict_selection.items():
    if j > threshold:
        features_selected.append(i)

In [47]:
import plotly.graph_objects as go
## Ploting the importance of features generated with polynomial function 
fig = go.Figure()
fig.add_trace(
        go.Bar(
            x = [x for x in train_eng.drop(["Ground_truth"], axis=1)],
            y = importance
                    ))
fig.update_layout(barmode='stack', xaxis={'categoryorder':'total descending'},width=8000,
    height=800,title="The importance of polynomial features regarding Ground Truth")
fig.show()

### Random Forest

In [36]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_selection import SelectFromModel

In [37]:
X_train = train_eng.drop(['Ground_truth','Id'], axis=1)
y_train = train_eng['Ground_truth'].values

In [ ]:
sel = SelectFromModel(RandomForestRegressor(n_estimators = 100,n_jobs=-1,criterion ="mae",random_state=2022))##,min_samples_leaf =3, oob_score=True, 
sel.fit(X_train, y_train)

In [ ]:
selected_feat= X_train.columns[(sel.get_support())]
len(selected_feat)